### Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier

### Data Load

In [3]:
train = pd.read_csv('/content/drive/MyDrive/난임/train.csv').drop(columns=['ID'])
test = pd.read_csv('/content/drive/MyDrive/난임/test.csv').drop(columns=['ID'])

In [4]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### EDA

In [5]:
# 모든 열을 비교하여 값이 동일한 열을 그룹으로 묶어주는 함수
def find_identical_columns(df):
    identical_columns = []
    checked_columns = set()

    for i, col1 in enumerate(df.columns):
        if col1 in checked_columns:
            continue

        group = [col1]
        for col2 in df.columns[i+1:]:
            if df[col1].equals(df[col2]):
                group.append(col2)
                checked_columns.add(col2)

        if len(group) > 1:
            identical_columns.append(group)

    return identical_columns

# 동일한 열을 그룹으로 출력
identical_column_groups = find_identical_columns(train_data)

# 결과 확인
identical_column_groups

NameError: name 'train_data' is not defined

In [ ]:
# 중복 컬럼 그룹 예시 (실제 결과를 반영하여 생성)
duplicate_columns = identical_column_groups

# 함수: 중복되는 컬럼을 제거하고 하나의 컬럼만 남김
def drop_duplicate_columns(df, duplicate_columns):
    for group in duplicate_columns:
        # 그룹 내 첫 번째 컬럼만 남기고 나머지 컬럼 삭제
        columns_to_drop = group[1:]  # 첫 번째 컬럼을 제외한 나머지
        df = df.drop(columns=columns_to_drop)
    return df

# 중복 컬럼 제거
train_data = drop_duplicate_columns(train_data, duplicate_columns)

# 결과 확인
train_data.columns.tolist()  # 남은 컬럼 목록 확인

In [ ]:
# 오브젝트 타입인 열 선택
object_columns = train_data.select_dtypes(include=['object'])

# 각 열의 유니크한 값 출력
for column in object_columns:
    unique_values = object_columns[column].unique()
    print(f"Column: {column}")
    print(f"Unique Values: {unique_values}\n")

In [ ]:
# 오브젝트 타입의 열 선택
object_columns = train_data.select_dtypes(include=['object'])

# 각 오브젝트 타입 열의 null 개수 출력
for column in object_columns:
    null_count = train_data[column].isnull().sum()
    if null_count > 0:
        print(f"Column: {column}")
        print(f"Number of null values: {null_count}\n")

In [ ]:
# float 및 int 타입의 열 선택
numeric_columns = train_data.select_dtypes(include=['float64', 'int64'])

# 각 수치형 열의 null 개수 출력
for column in numeric_columns:
    null_count = train_data[column].isnull().sum()
    if null_count > 0:
        print(f"Column: {column}")
        print(f"Number of null values: {null_count}\n")

### Data Pre-processing

In [ ]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [ ]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [ ]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [ ]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [ ]:
X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

### Train

In [ ]:
model = ExtraTreesClassifier(random_state=42)

model.fit(X_train_encoded, y)

### Predict

In [ ]:
pred_proba = model.predict_proba(X_test_encoded)[:, 1]

### Submission

In [ ]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = pred_proba

In [ ]:
sample_submission.to_csv('./baseline_submit.csv', index=False)